In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch_tabnet

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import datetime
import random
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from scipy.sparse import csc_matrix
from pytorch_tabnet import tab_network
from pytorch_tabnet.tab_network import TabNet
from pytorch_tabnet.utils import create_sampler, PredictDataset, create_explain_matrix

from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from keras.preprocessing.sequence import pad_sequences

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score


np.random.seed(2021)
torch.manual_seed(2021)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# 텍스트와 정형 데이터 불러오기


In [ ]:
path1 = "/content/drive/My Drive/LG/UPLUS/"
path2 = "drive/My Drive/NLP_practice/Blue-House/"

In [ ]:
 train1 = pd.read_csv(path1 + 'Stars.csv')
train2 = pd.read_csv(path2 + 'train.csv')

In [ ]:
train2 = train2.iloc[:train1.shape[0],:]
train = pd.concat([train2.data, train1], axis = 1)

In [ ]:
X = train.loc[:,train.columns != 'Type']
y = train.loc[:, 'Type']

target = 'Type'

In [ ]:
bert_X = X.loc[:, X.columns == 'data']
tab_X = X.loc[:,X.columns != 'data']

In [ ]:
bert_X

,data
0,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...
...,...
235,대한민국의 공교육을 살려주십시요!!!. 오늘 충격적인 학부모 연수가 중학교 강당에서...
236,헌법법폐지하라. 문재인대통령 님\n헌법폐지해주세요\n헌법이통과되면 이나라는 매우 ...
237,KBS 드라마 광고 중간삽입은 좋다. 하지만 수신료는 받지 마라. 이번 KBS 월화...
238,베이비시터의 자격여부 나라에서 승인하십시오. 요양보호사조차 여러자격요건과 실습이 필...


# 전처리 작업

In [ ]:
# Tokenizer 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
# TEXT 데이터 변환
sentences = bert_X['data']
def sentence_clean(text):
    # 스페셜 토큰 추가
    text = ["[CLS] " + str(sentence) + " [SEP]" for sentence in text]
    # 문장 단위"\\n" 제거
    text = [sentence.replace('\\n', '.') for sentence in text]
    text = [sentence.replace('.', ' ') for sentence in text]
    text = [sentence.replace('!', ' ') for sentence in text]
    text = [sentence.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") for sentence in text]
    return(text)

sentences = sentence_clean(sentences)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
# print(tokenized_texts[0])

In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 256

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(len(tokenizer.convert_tokens_to_ids(tokenized_texts[0])))

281


In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [ ]:
# 훈련셋과 검증셋으로 분리
# train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
#                                                                                     labels, 
#                                                                                     random_state=7, 
#                                                                                     test_size=0.2)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
# train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
#                                                        input_ids,
#                                                        random_state=7, 
#                                                        test_size=0.2)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(input_ids)
train_labels = torch.tensor(y.values)
train_masks = torch.tensor(attention_masks)
# validation_inputs = torch.tensor(validation_inputs)
# validation_labels = torch.tensor(validation_labels.values)
# validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
# print(validation_inputs[0])
# print(validation_labels[0])
# print(validation_masks[0])

In [ ]:
cvt_tab_X = tab_X.copy()
cvt_tab_X.loc[:,['Color', 'Spectral_Class']] = cvt_tab_X.loc[:,['Color', 'Spectral_Class']].astype('category')

In [ ]:
# TabNet 숫자 변수 스케일링
std_scaler = StandardScaler()
cvt_tab_X.loc[:,cvt_tab_X.columns[cvt_tab_X.dtypes != 'category'].tolist()] = std_scaler.fit_transform(cvt_tab_X.loc[:,cvt_tab_X.columns[cvt_tab_X.dtypes != 'category'].tolist()])

In [ ]:
# TabNet 카테고리 변수 인코딩
categorical_columns = []
categorical_dims = {}
for col in tqdm(cvt_tab_X.columns[cvt_tab_X.dtypes == 'category']):
    l_enc = LabelEncoder()
    cvt_tab_X[col] = l_enc.fit_transform(cvt_tab_X[col].values)
    categorical_columns.append(col)
    categorical_dims[col] = len(l_enc.classes_)


unused_feat = []

features = [col for col in cvt_tab_X.columns if col not in unused_feat+[target]]

cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [ ]:
# 배치 사이즈
batch_size = 10

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels, torch.tensor(cvt_tab_X.values))
# train_sampler = RandomSampler(train_data) sampler=train_sampler, 
train_dataloader = DataLoader(train_data, batch_size=batch_size)

# validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
# validation_sampler = SequentialSampler(validation_data)
# validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# Network 정의

In [ ]:
# Bert 객체 지정
class BERT_Module(nn.Module):

    def __init__(self, BertModel, pretrain_options=None, out_attention = False):
        super(BERT_Module, self).__init__()

        if not pretrain_options:
            pretrain_options = "bert-base-multilingual-cased"
        self.encoder = BertModel.from_pretrained(pretrain_options, output_attentions = out_attention)

    def forward(self, input_ids=None, token_type_ids=None, attention_mask=None):
        bert_output = self.encoder(input_ids = input_ids,
                                   token_type_ids = token_type_ids,
                                   attention_mask = attention_mask)

        return bert_output

In [ ]:
# TabNet 객체 지정
class TabNet_Module(nn.Module):
    def __init__(self, TabNet, input_dim, output_dim, cat_idxs, cat_dims):
        super(TabNet_Module, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.cat_idxs = cat_idxs
        self.cat_dims = cat_dims
        self.Tabnetwork = TabNet(input_dim = self.input_dim,
                                 output_dim = self.output_dim,
                                 cat_idxs = self.cat_idxs,
                                 cat_dims = self.cat_dims)
        
    def forward(self, x):
        steps_output, M_loss = self.Tabnetwork(x)
        
        return steps_output, M_loss
        
    def forward_tab_masks(self, x, batch_nb, res_explain, res_masks): # tabnet attention 산출을 위한 함수
        self.reducing_matrix = create_explain_matrix(
            self.Tabnetwork.input_dim,
            self.Tabnetwork.cat_emb_dim,
            self.Tabnetwork.cat_idxs,
            self.Tabnetwork.post_embed_dim,
        )
        
        M_explain, masks = self.Tabnetwork.forward_masks(x)
        for key, value in masks.items():
            masks[key] = csc_matrix.dot(
                value.cpu().detach().numpy(), self.reducing_matrix
            )

        res_explain.append(
            csc_matrix.dot(M_explain.cpu().detach().numpy(), self.reducing_matrix)
        )

        if batch_nb == 0:
            res_masks = masks
        else:
            for key, value in masks.items():
                res_masks[key] = np.vstack([res_masks[key], value])
                                
        return res_explain, res_masks # res_explain을 vstack하여 attention을 산출

In [ ]:
# MLP 객체 지정
class MLP_Module(nn.Module):
    def __init__(self, input_dim, num_class):
        super(MLP_Module, self).__init__()
        
        self.layer_1 = nn.Linear(input_dim, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        return x

In [ ]:
# https://colab.research.google.com/drive/1SyRrBAudJHiKjHnxXaZT5w_ukA0BmK9X?usp=sharing
# Ensemble 모델 객체 지정
class TabBertEnsemble(nn.Module):
    def __init__(self, BERT_Module, TabNet_Module, MLP_Module):
        super().__init__()
        self.bert_model = BERT_Module
        self.tabnet = TabNet_Module
        self.classifier = MLP_Module

    def forward(
            self,
            bert_x =None,
            attention_mask=None,
            token_type_ids=None,
            tabnet_x = None
    ):
        bert_output = self.bert_model(bert_x,
                                      token_type_ids=token_type_ids,
                                      attention_mask=attention_mask)
        
        tabnet_output, M_loss = self.tabnet(tabnet_x)

        # Bert [CLS] 토큰의 마지막 hidden layer와 Tabnet 결과를 concat하여 mlp로 전달
        concated_output = torch.cat([bert_output[1], tabnet_output], dim=1)
        final_output = self.classifier(concated_output)
        return final_output

In [ ]:
# 각 객체 모듈 초기화
bert_model = BERT_Module(BertModel)                         # 훈련시 attention 저장 비활성화

tabnet_model = TabNet_Module(TabNet
                             ,input_dim = tab_X.shape[1]
                             ,output_dim = 768              # Bert Hidden layer 차원과 동일하게 output 차원 지정
                             ,cat_idxs = cat_idxs
                             ,cat_dims = cat_dims)

MLP_cls = MLP_Module(input_dim = 768 * 2, num_class = 6)

ensemble_model = TabBertEnsemble(BERT_Module = bert_model,
                         TabNet_Module = tabnet_model,
                         MLP_Module = MLP_cls)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Ensemble Network 학습

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(ensemble_model.parameters(),
                  lr = 1e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# loss func 설정
criterion = nn.CrossEntropyLoss()

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))

    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# GPU 체크
print(torch.cuda.is_available())
device_num = torch.cuda.current_device()
ensemble_model.cuda(device_num)

In [ ]:
print("======== Begin training ========")
print("")

# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 성능 지표 초기화
loss_stats = {'train':[],'val':[]}
accuracy_stats = {'train':[],'val':[]}
recall_stats = {'train':[],'val':[]}
precision_stats = {'train':[],'val':[]}

ensemble_model.zero_grad() # 그래디언트 초기화

for e in tqdm(range(1, epochs+1)):
    # ========================================
    #               Training
    # ========================================

    # Epoch 훈련 성능 지표 초기화
    train_epoch_loss = 0
    train_epoch_accuracy = 0
    train_epoch_recall = 0
    train_epoch_precision = 0

    t0 = time.time()                                        # 시작 시간 설정

    ensemble_model.train()                                  # 훈련모드로 변경

    for i, train_batch in enumerate(train_dataloader):      # 텍스트, 정형 데이터 로더 동시 진행
        # 배치를 GPU에 넣음
        train_batch = tuple(t.cuda(device_num) for t in train_batch)
        train_text_data, train_attention_mask, train_labels, train_tab_data = train_batch
        optimizer.zero_grad()                               # 그래디언트 초기화

        y_train_pred = ensemble_model(bert_x = train_text_data,
                              attention_mask = train_attention_mask,
                              tabnet_x = train_tab_data)
        # print(np.shape(y_train_pred))
        # print(np.shape(train_labels))
        train_loss = criterion(y_train_pred, train_labels)  # 로스 산출
        
        train_loss.backward()                               # 로스 기반으로 gradient 계산
        optimizer.step()                                    # 역전파 수행
        
        train_epoch_loss += train_loss.item()               # 배치내 로스 합산

        # 분류별 훈련단계 평균 모델 성능치 계산
        _, train_predictions = torch.max(y_train_pred, 1)
        # 예측값과 실제값을 CPU로 이동 후 성능 계산
        train_predictions = train_predictions.cpu().detach().numpy()
        train_labels = train_labels.cpu().detach().numpy()
        train_epoch_accuracy += accuracy_score(train_labels, train_predictions)
        train_epoch_recall += recall_score(train_labels, train_predictions, average='macro', zero_division = 0)
        train_epoch_precision += precision_score(train_labels, train_predictions, average='macro', zero_division = 0)
        
    # ========================================
    #               Validation
    # ========================================   
    with torch.no_grad(): # 검증 단계는 gradient 비활성화
        # Epoch 검증 성능 지표 초기화
        val_epoch_loss = 0
        val_epoch_accuracy = 0
        val_epoch_recall = 0
        val_epoch_precision = 0
        
        ensemble_model.eval()                                # 검증모드로 변경

        for i, val_batch in enumerate(val_dataloader):
            # 배치를 GPU에 넣음
            val_batch = tuple(t.cuda(device_num) for t in val_batch)
            val_text_data, val_attention_mask, val_labels, val_tab_data = val_batch
            
            y_val_pred = ensemble_model(bert_x = val_text_data,
                                attention_mask = val_attention_mask,
                                tabnet_x = val_tab_data)
                        
            val_loss = criterion(y_val_pred, val_labels)    # 로스 산출
            
            val_epoch_loss += val_loss.item()               # 배치내 로스 합산

            # 분류별 검증단계 평균 모델 성능치 계산
            _, val_predictions = torch.max(y_val_pred, 1)
            # 예측값과 실제값을 CPU로 이동 후 성능 계산
            val_predictions = val_predictions.cpu().detach().numpy()
            val_labels = val_labels.cpu().detach().numpy()
            val_epoch_accuracy += accuracy_score(val_labels, val_predictions)
            val_epoch_recall += recall_score(val_labels, val_predictions, average='macro', zero_division = 0)
            val_epoch_precision += precision_score(val_labels, val_predictions, average='macro', zero_division = 0)

    # Epoch내 모델 성능 종합
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_accuracy/len(train_loader))
    accuracy_stats['val'].append(val_epoch_accuracy/len(val_loader))
    recall_stats['train'].append(train_epoch_recall/len(train_loader))
    recall_stats['val'].append(val_epoch_recall/len(val_loader))
    precision_stats['train'].append(train_epoch_precision/len(train_loader))
    precision_stats['val'].append(val_epoch_precision/len(val_loader))
    
    end_time = time.time()
    print(f'Epoch {e+0:03} elapsed time {format_time(end_time - t0)}: | Train Loss: {train_epoch_loss/len(train_dataloader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Accuracy: {train_epoch_accuracy/len(train_loader):.3f}| Val Accuracy: {val_epoch_accuracy/len(val_loader):.3f} | Train Recall: {train_epoch_recall/len(train_loader):.3f}| Val Accuracy: {val_epoch_recall/len(val_loader):.3f} | Train Precision: {train_epoch_precision/len(train_loader):.3f}| Val Accuracy: {val_epoch_precision/len(val_loader):.3f} |')

print("")
print("======== Training complete! ========")

# Attention 추출 및 테스트셋 예측

In [ ]:
# Attention 추출 활성화
bert_model = BERT_Module(BertModel, out_attention=True) # 저장 된 Bert 모델 불러온 후, attention 저장 활성화
bert_model.cuda(device_num)
tabnet_model.cuda(device_num)

In [ ]:
bert_attention_result = []
for i, batch in enumerate(train_dataloader):
    batch = tuple(t.cuda(device_num) for t in batch[:2])
    train_text_data, train_attention_mask, train_labels, _ = batch
    bert_output = bert_model(input_ids = train_text_data, attention_mask = train_attention_mask)
    bert_attention_result.append(bert_output[-1][-1][:,0,:,0].cpu().detach().numpy()) # 마지막 layer의 마지막 attention head의 [cls] token attention 추출
bert_attention_result = np.vstack(bert_attention_result)
print(np.shape(bert_attention_result))

(240, 256)


In [ ]:
# TabNet Attention 추출
res_explain = []
res_masks = None
for i, batch in enumerate(train_dataloader):
    train_tab_data = batch[3].cuda(device_num)
    res_explain, res_masks = tabnet_model.forward_tab_masks(train_tab_data, batch_nb = i, res_explain = res_explain, res_masks = res_masks)
res_explain = np.vstack(res_explain)
print(np.shape(res_explain))

(240, 6)


## Multi GPU Code sample

In [ ]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)

model.to(device)

for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

In [ ]:
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel


def main():
    args = parser.parse_args()

    ngpus_per_node = torch.cuda.device_count()
    args.world_size = ngpus_per_node * args.world_size
    mp.spawn(main_worker, nprocs=ngpus_per_node, 
             args=(ngpus_per_node, args))
    
    
def main_worker(gpu, ngpus_per_node, args):
    global best_acc1
    args.gpu = gpu
    torch.cuda.set_device(args.gpu)
    
    print("Use GPU: {} for training".format(args.gpu))
    args.rank = args.rank * ngpus_per_node + gpu
    dist.init_process_group(backend='nccl', 
                            init_method='tcp://127.0.0.1:FREEPORT',
                            world_size=args.world_size, 
                            rank=args.rank)
    
    model = Bert()
    model.cuda(args.gpu)
    model = DistributedDataParallel(model, device_ids=[args.gpu])

    acc = 0
    for i in range(args.num_epochs):
        model = train(model)
        acc = test(model, acc)
        
        

from torch.utils.data.distributed import DistributedSampler

train_dataset = datasets.ImageFolder(traindir, ...)
train_sampler = DistributedSampler(train_dataset)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=False,
    num_workers=args.workers, pin_memory=True, sampler=train_sampler)